## Imports and reading in data

In [74]:
#imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import seaborn as sns
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier

In [59]:
#Paths to files
data_path = 'Q:/hackers09/shared/data/'
cancer_path = os.path.join(data_path, 'df_cancer.csv')
echo_path = os.path.join(data_path, 'df_echo.csv')
encounter_path = os.path.join(data_path, 'df_encounter.csv')
labs_path = os.path.join(data_path, 'df_labs.csv')
outcome_path = os.path.join(data_path, 'df_outcome.csv')
problist_path = os.path.join(data_path, 'df_problist.csv')
radiology_path = os.path.join(data_path, 'df_radiology.csv')
registry_path = os.path.join(data_path, 'df_registry.csv')
vitals_path = os.path.join(data_path, 'df_vitals.csv')

In [60]:
#Read in dataframes
cancer_df = pd.read_csv(cancer_path, encoding='ISO-8859-1')
cancer_df.set_index("HSP_ENC", inplace = True)
echo_df = pd.read_csv(echo_path, encoding='ISO-8859-1')
echo_df.set_index("HSP_ENC", inplace = True)
encounter_df = pd.read_csv(encounter_path, encoding='ISO-8859-1')
encounter_df.set_index("HSP_ENC", inplace = True)
labs_df = pd.read_csv(labs_path, encoding='ISO-8859-1')
labs_df.set_index("HSP_ENC", inplace = True)
outcome_df = pd.read_csv(outcome_path, encoding='ISO-8859-1')
outcome_df.set_index("HSP_ENC", inplace = True)
problist_df = pd.read_csv(problist_path, encoding='ISO-8859-1')
problist_df.set_index("HSP_ENC", inplace = True)
radiology_df = pd.read_csv(radiology_path, encoding='ISO-8859-1')
radiology_df.set_index("HSP_ENC", inplace = True)
registry_df = pd.read_csv(registry_path, encoding='ISO-8859-1')
registry_df.set_index("HSP_ENC", inplace = True)
vitals_df = pd.read_csv(vitals_path, encoding='ISO-8859-1')
vitals_df.set_index("HSP_ENC", inplace = True)

##  
## Define functions to merge datasets

### Cancer

In [61]:
#function to merge cancer data
def merge_cancer(enc_df, cnc_df):
    #Make modifications to cancer df
    mod_cancer_df = cnc_df.copy()
    mod_cancer_df.reset_index(level=0, inplace=True)
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Missing Remission Date', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Previously Positive', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not Documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown, Not documented', 'cancer_at_enc'] = 'Unknown'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'No Cancer', 'cancer_at_enc'] = '1'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Unknown', 'cancer_at_enc'] = '2'
    mod_cancer_df.loc[mod_cancer_df.cancer_at_enc == 'Cancer', 'cancer_at_enc'] = '3'
    mod_cancer_df['cancer_at_enc'] = mod_cancer_df['cancer_at_enc'].astype(int)
    mod_cancer_df.drop_duplicates(['PATIENT_ID', 'HSP_ENC'])
    
    #Take only cancer status column with max value
    drop_cancer_df = mod_cancer_df[['HSP_ENC', 'cancer_at_enc']]
    drop_cancer_df = drop_cancer_df.groupby('HSP_ENC',group_keys=False).apply(lambda x: x.loc[x['cancer_at_enc']==x['cancer_at_enc'].max()])

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, drop_cancer_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')

    #Replace NaN in encounter df with 0 (Never had cancer)
    mergeRes['cancer_at_enc'].fillna(0, inplace=True)
    
    #Renaming the cancer column
    mergeRes.rename(columns={'cancer_at_enc': 'CANCER_RANK'}, inplace=True)
    
    mergeRes.reset_index(drop=True)
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Registry

In [62]:
def merge_registry(enc_df, reg_df):
    reg_list = reg_df.REGISTRY_NAME.unique().tolist()
    reg_df_copy = reg_df.copy()
    reg_df_copy.reset_index(level=0, inplace=True)

    tst_df = reg_df_copy[['HSP_ENC']].copy()
    for item in reg_list:
        tst_df[item] = False
    tst_df = tst_df.drop_duplicates()


    for index, row in reg_df_copy.iterrows():
        enc_id = reg_df_copy.iloc[index, 0]
        curr_reg = reg_df_copy.iloc[index, 2]
        tst_df.loc[tst_df['HSP_ENC'] == enc_id, [curr_reg]] = True

    #Merge with encounter df and drop dups
    mergeRes = pd.merge(enc_df, tst_df, on='HSP_ENC', how='left')
    mergeRes = mergeRes.drop_duplicates('HSP_ENC')
    
    #Replace NaN in encounter df with False, no record
    for item in reg_list:
        mergeRes[item].fillna(False, inplace=True)
    
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Vitals

In [63]:
def merge_vitals(enc_df, data_path):
    vital_data_path = os.path.join(data_path, 'vitals.csv')
    vital_data_df = pd.read_csv(vital_data_path, encoding='ISO-8859-1')
    vital_data_df = vital_data_df.drop('Unnamed: 0', 1)
    
    mergeRes = pd.merge(enc_df, vital_data_df, on='HSP_ENC', how='left')
    mergeRes.set_index("HSP_ENC", inplace = True)
    return mergeRes

### Radiology

In [82]:
def merge_encounter_radiology(df_encounter, df_radiology, cutoff_OrderTime=12, cutoff_ED_Disp=12):
    # generate a new df_CT dataframe
    # 1. focus on 'CT ANGIOGRAM' only
    # 2. order time within 12h
    # 3. keep only the first order for the outliers (only one data point that has 2 orders)

    df_CT = df_radiology[df_radiology['NAME'].apply(lambda x: x.startswith('CT AN'))]
    df_CT = df_CT[df_CT['ORDER_TIME_DIFFSEC'] <= cutoff_OrderTime*3600]
    df_CT = df_CT.drop_duplicates('HSP_ENC', keep='first')

    # combine df_encounter and df_CT based on 'HSP_ENC' id
    df_encounter = df_encounter[df_encounter['ED_DISP_TIME_DIFFSEC']<=cutoff_ED_Disp*3600]
    df_enc_CT = pd.merge(df_encounter, df_CT, how='left', on='HSP_ENC')
    df_enc_CT.set_index('HSP_ENC', inplace=True)
    
    return df_enc_CT

In [83]:
def expand_outcome(dfOutcome):
    ## Outcome expansion   
    
    # Drop any outcomes after 48 hr
    dfOutcome = dfOutcome[dfOutcome['ORDER_TIME_DIFFSEC']<48*60*60]
    
    # Create columns for each unique outcome
    lsExpandedOutcomeCols = []
    lsOutcomeColsToRetain = dfOutcome.columns[2:].tolist()
    lsUniqueOutcomes = dfOutcome['name_gen'].value_counts().index.tolist()
    for strOutcome in lsUniqueOutcomes:
        lsExpandedOutcomeCols += [strCol + '_' + strOutcome for strCol in lsOutcomeColsToRetain]

    dfExpandedOutcome = pd.DataFrame(columns=lsExpandedOutcomeCols+['PATIENT_ID'])
    dfExpandedOutcome['HSP_ENC'] = dfOutcome['HSP_ENC'].value_counts().index
    dfExpandedOutcome = dfExpandedOutcome.set_index('HSP_ENC')

    for nEnc in dfOutcome['HSP_ENC'].value_counts().index:
        for nIdx in dfOutcome[dfOutcome['HSP_ENC']==nEnc].index:
            lsTempCols = [strCol + '_' + dfOutcome.at[nIdx, 'name_gen'] for strCol in lsOutcomeColsToRetain]
            lsTempCols.append('PATIENT_ID')
            dfExpandedOutcome.loc[nEnc, lsTempCols] = dfOutcome.loc[nIdx, lsOutcomeColsToRetain+['PATIENT_ID']].values

    # Create boolean column for order time < 48 hrs for any outcome
    lsOrderTimeCols = [strCol for strCol in dfExpandedOutcome.columns if 'ORDER_TIME' in strCol]

    dfExpandedOutcome['b48hr'] = np.zeros(dfExpandedOutcome.shape[0])
    for nEnc in dfExpandedOutcome.index:
        for nVal in dfExpandedOutcome.loc[nEnc, lsOrderTimeCols].values:
            if nVal < 172800:
                dfExpandedOutcome.at[nEnc, 'b48hr'] = 1

    # Column for minimum order time from all outcomes
    dfExpandedOutcome['MinOrderTime'] = dfExpandedOutcome.loc[:, lsOrderTimeCols].min(axis=1)
    dfExpandedOutcome = dfExpandedOutcome.sort_values('MinOrderTime')
    
    return dfExpandedOutcome


def compile_echo(dfEcho):    
    # Keep only 12 hr echos
    dfEcho = dfEcho[dfEcho['ORDER_INST_DIFFSEC'] < 12*60*60]

    # Add all narratives together
    lsUniqueEchoEnc = dfEcho['HSP_ENC'].value_counts().index.tolist()

    lsEchoCols = dfEcho.columns.tolist()
    lsEchoCols.remove('new_line')
    lsEchoCols.remove('NARRATIVE')
    dfCompiledEcho = pd.DataFrame(columns=lsEchoCols)
    dfCompiledEcho['HSP_ENC'] = lsUniqueEchoEnc
    dfCompiledEcho = dfCompiledEcho.set_index('HSP_ENC')
    lsEchoCols.remove('HSP_ENC')

    lsCompiledNarratives = []
    for nEnc in lsUniqueEchoEnc:
        lsUniqueEchoOrderId = dfEcho[dfEcho['HSP_ENC']==nEnc]['ORDER_PROC_ID'].value_counts().index.tolist()
        #if len(lsUniqueEchoOrderId)>1:
            #print(nEnc)
        nFirstEchoOrderId = lsUniqueEchoOrderId[0] # Keep first one only
        
        # Compile echo data
        strCompiled = dfEcho[dfEcho['ORDER_PROC_ID']==nFirstEchoOrderId]['NARRATIVE'].str.cat(sep=' ')
        dfCompiledEcho.loc[nEnc, lsEchoCols] = dfEcho[dfEcho['ORDER_PROC_ID']==nFirstEchoOrderId][lsEchoCols].iloc[0]
        lsCompiledNarratives.append(strCompiled)
    dfCompiledEcho['NARRATIVE_compiled'] = lsCompiledNarratives

    # Keep only 12 hr echos
    dfCompiledEcho = dfCompiledEcho[dfCompiledEcho['ORDER_INST_DIFFSEC'] < 12*60*60]
    
    return dfCompiledEcho

def get_merge_dfs(dfOutcome, dfEcho, dfEncounter):
    dfExpandedOutcome = expand_outcome(dfOutcome)
    dfCompiledEcho = compile_echo(dfEcho)
    dfEncounter = dfEncounter.set_index('HSP_ENC')
    
    dfMerged = dfEncounter.merge(dfExpandedOutcome, how='left', on='HSP_ENC')
    dfMerged = dfMerged.merge(dfCompiledEcho, how='left', on='HSP_ENC')
    
    return dfMerged

# Looks like this is working now :)

In [85]:
def merge_echo(out_df, ech_df, enc_df, rad_df):
    # read each dataset
    mod_out_df = out_df.copy()
    mod_out_df.reset_index(level=0, inplace=True)
    mod_ech_df = ech_df.copy()
    mod_ech_df.reset_index(level=0, inplace=True)
    mod_enc_df = enc_df.copy()
    mod_enc_df.reset_index(level=0, inplace=True)
    mod_rad_df = rad_df.copy()
    mod_rad_df.reset_index(level=0, inplace=True)
    
    df_enc_echo_outcome = get_merge_dfs(mod_out_df, mod_ech_df, mod_enc_df)
    df_enc_CT = merge_encounter_radiology(mod_enc_df, mod_rad_df)

    # clean the dataset
    df_cleaned = df_enc_echo_outcome.merge(df_enc_CT, how='left', on='HSP_ENC')

    # raw column names
    name_gen = []
    for i in df_cleaned.columns.values:
        if i.startswith('name_gen') or i.startswith('NARR'):
            name_gen.append(i)
    name_gen = sorted(name_gen) + ['b48hr']

    # rename the column names
    df_cleaned = df_cleaned[name_gen]
    new_columns = ['CT', 'Echo', 'CPR', 'DEATH', 'INTUBATION', 'PPV', 'THROMBOLYSIS', 'THROMBOLYSIS_PROC', 'VASOPRESSORS', 'B48hr']
    df_cleaned.columns = new_columns

    # fill Nan
    df_cleaned['CT'].fillna('No_CT', inplace=True)
    df_cleaned['Echo'].fillna('No_Echo', inplace=True)
    df_cleaned.fillna(0, inplace=True)

    # convert to 1
    map_dict = {'DEATH': 1, 
                'INTUBATION': 1,
                'PPV': 1,
                'THROMBOLYSIS': 1,
                'THROMBOLYSIS_PROC': 1,
                'VASOPRESSORS': 1,
                0:0}
    for col in ['DEATH', 'INTUBATION', 'PPV', 'THROMBOLYSIS', 'THROMBOLYSIS_PROC', 'VASOPRESSORS']:
        df_cleaned[col] = df_cleaned[col].map(map_dict)
        
    return df_cleaned

##  
## Start calling merge functions

In [64]:
encounter_df = merge_registry(encounter_df, registry_df)

In [66]:
encounter_df = merge_vitals(encounter_df, data_path)

In [ ]:
encounter_df = merge_echo(outcome_df, echo_df, encounter_df, radiology_df)

In [70]:
data = encounter_df

##  
## Preprocess / Normalize the merged data

In [ ]:
y = data.iloc[:, -1]
X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.7, random_state=0)

##  
##  Define params and run gridsearch to find best params

In [71]:
params = {
    'n_estimators': [5, 10, 15],
    'criterion': ['gini', 'entropy'],
    'max_depth': [10, None],
    'min_samples_split': [2, 4]
}

In [76]:
clf = GridSearchCV(RandomForestClassifier(), params, cv=5, scoring = 'roc_auc')
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print(clf.best_params_)
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))

NameError: name 'X_train' is not defined

##  
## Testing stuff